<a href="https://colab.research.google.com/github/hrishikesav/BigData/blob/master/CalculateAverageUsingCombiner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This program calculates average of input numbers using Combiner**

In [2]:
# Run and print a shell command.
def run(cmd):
  print('>> {}'.format(cmd))
  !{cmd}
  print('')

# Install apache-beam.
run('pip install --quiet apache-beam')

>> pip install --quiet apache-beam



In [10]:
import apache_beam as beam

class AverageFn(beam.CombineFn):

  def create_accumulator(self):
    return(0.0,0)                 # to initialize to (sum,total)

  def add_input(self,sum_count,input_value):
    (sum,count) = sum_count
    return sum+input_value, count+1

  def merge_accumulators(self,accumulators):
    sums,counts = zip(*accumulators)
    return sum(sums), sum(counts)

  def extract_output(self,sum_count):
    (sum,count) = sum_count
    return sum/count if count else float('NaN')

p = beam.Pipeline()

avg = (
    p
    | "Create list" >> beam.Create([1,2,3,4,5])
    | "Get avg" >> beam.CombineGlobally(AverageFn())
    | "Write ouput" >> beam.io.WriteToText('data/comb')
)

p.run()

!{('head -n 20 data/comb-00000-of-00001')}


3.0


**Creating your own Composite transform class**

In [12]:
import apache_beam as beam

class MyCustomTransform(beam.PTransform):
  def expand(self,input):
    trans = (
        input
        | "Create list" >> beam.Create([1,2,3,4,5,6])
        | "Get avg" >> beam.CombineGlobally(AverageFn())
        | "Write ouput" >> beam.io.WriteToText('data/comb-custom')
    )
    return trans

class AverageFn(beam.CombineFn):

  def create_accumulator(self):
    return(0.0,0)                 # to initialize to (sum,total)

  def add_input(self,sum_count,input_value):
    (sum,count) = sum_count
    return sum+input_value, count+1

  def merge_accumulators(self,accumulators):
    sums,counts = zip(*accumulators)
    return sum(sums), sum(counts)

  def extract_output(self,sum_count):
    (sum,count) = sum_count
    return sum/count if count else float('NaN')

p = beam.Pipeline()

avg = (
    p
    | "Calling My Custom transform" >> MyCustomTransform()
)

p.run()

!{('head -n 20 data/comb-custom-00000-of-00001')}


3.5
